In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    matthews_corrcoef,
    confusion_matrix,
    precision_recall_curve,
    roc_curve,
)
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import label_binarize

# Load the dataset from a CSV file
dataset_path = r'C:\Users\tahsi\OneDrive\Desktop\python_ws\Feature Selection\L1 Regularization\selected_dataset.csv'
df = pd.read_csv(dataset_path)

# Separate the features and target variable
X = df.drop('classification', axis=1)
y = df['classification']

# Gather additional data (assuming you have another dataset stored in 'additional_dataset.csv')
additional_dataset_path = r'C:\Users\tahsi\OneDrive\Desktop\python_ws\Feature Selection\L1 Regularization\dataset_NoPCA.csv'
additional_df = pd.read_csv(additional_dataset_path)
# Concatenate additional data with existing data
X = pd.concat([X, additional_df.drop('classification', axis=1)])
y = pd.concat([y, additional_df['classification']])

# Handle missing values
imputer = SimpleImputer(strategy='mean')  # Replace NaN values with column mean
X = imputer.fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the first classifier (Random Forest)
clf1 = RandomForestClassifier(n_estimators=100, random_state=42)
clf1.fit(X_train, y_train)

# Train the second classifier (Gradient Boosting)
clf2 = GradientBoostingClassifier(n_estimators=50, random_state=42)
clf2.fit(X_train, y_train)

# Make predictions using both classifiers
pred1 = clf1.predict(X_test)
pred2 = clf2.predict(X_test)

# Combine predictions using plurality voting
combined_pred = np.array([max(set([pred1[i], pred2[i]]), key=list([pred1[i], pred2[i]]).count) for i in range(len(pred1))])

# Calculate accuracy, precision, recall, F1 score, AUC, and V score of the combined predictions
accuracy = accuracy_score(y_test, combined_pred)
precision = precision_score(y_test, combined_pred, average='weighted', zero_division=0)
recall = recall_score(y_test, combined_pred, average='weighted', zero_division=0)
f1 = f1_score(y_test, combined_pred, average='macro')
auc = roc_auc_score(label_binarize(y_test, classes=np.unique(y)), label_binarize(combined_pred, classes=np.unique(y)), average='macro')
v_score = matthews_corrcoef(y_test, combined_pred)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("AUC:", auc)
print("V Score:", v_score)

# Calculate and visualize the confusion matrix
cm = confusion_matrix(y_test, combined_pred)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, cmap='Blues', fmt='d')
plt.title('Confusion Matrix')
plt.show()

# Define class_names based on unique values in the target variable
class_names = np.unique(y)

# Create a histogram for the evaluation metrics
metrics = ['Accuracy', 'Precision', 'Recall', 'F1 Score', 'AUC', 'V Score']
values = [accuracy, precision, recall, f1, auc, v_score]

plt.figure(figsize=(10, 6))
bars = plt.bar(metrics, values)
plt.xlabel('Metrics')
plt.ylabel('Score')
plt.title('Evaluation Metrics For RF+GB+L1 Regularization Algorithm')

# Add text annotations to the bars
for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() / 2, yval, f'{yval:.2%}', ha='center', va='bottom')

plt.show()

# Plot the ROC curve
plt.figure(figsize=(8, 6))
for i, class_name in enumerate(class_names):
    fpr, tpr, _ = roc_curve(label_binarize(y_test, classes=np.unique(y))[:, i], label_binarize(combined_pred, classes=np.unique(y))[:, i])
    plt.plot(fpr, tpr, label='ROC curve of class {0} (area = {1:0.2f})'.format(class_name, auc))

plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc="lower right")
plt.show()

# Plot the class-wise precision and recall with percentage numbers
plt.figure(figsize=(8, 6))
bar_width = 0.35
index = np.arange(len(class_names))

# Plot precision bars
plt.bar(index, precision, bar_width, label='Precision', color=precision_color)

# Plot recall bars with different color
plt.bar(index, recall, bar_width, label='Recall', color=recall_color, alpha=0.7)

plt.xlabel('Class')
plt.ylabel('Score')
plt.title('Class-wise Precision and Recall')
plt.xticks(index, class_names)
plt.legend()

# Add percentage numbers on top of the bars
for i, bar in enumerate(plt.bar(index, precision, bar_width)):
    plt.text(bar.get_x() + bar.get_width() / 2, bar.get_height(),
             f'{bar.get_height():.2%}', ha='center', va='bottom', color='white')

for i, bar in enumerate(plt.bar(index, recall, bar_width, alpha=0.7)):
    plt.text(bar.get_x() + bar.get_width() / 2, bar.get_height(),
             f'{bar.get_height():.2%}', ha='center', va='bottom', color='black')

plt.show()

# Print class-wise precision and recall
for class_index, class_name in enumerate(class_names):
    print(f"Class {class_name}:")
    print(f"   Precision: {precision:.2%}")
    print(f"   Recall: {recall:.2%}")
    print()


ModuleNotFoundError: No module named 'pandas'